Для первого раза надо будет раскомментить и скачать список стоп слов

In [44]:
import time
import pandas as pd
import numpy as np
import scipy.spatial as ss
import sklearn.cluster as sc
import sklearn.manifold as sm
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as smt
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Plotting config
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/Alex_Reznikova/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'randint']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [45]:
train_data = pd.read_csv('train_groups.csv', encoding='utf-8')
# train_data
train_groups = train_data.group_id
train_y = train_data.target

test_data = pd.read_csv('test_groups.csv', encoding='utf-8')
test_groups = test_data.group_id

In [46]:
'Parse_done/jackard_titles_lemmatization_train_not_nan.csv'
'Parse_done/jackard_titles_lemmatization_test_not_nan.csv'

'Parse_done/jackard_titles_train_not_nan.csv'
'Parse_done/jackard_titles_test_not_nan.csv'

'Parse_done/jackard_texts_train_raw_ALL.csv'
'Parse_done/jackard_texts_test_raw_ALL.csv'

'Parse_done/jackard_texts_train_lemm_char_ALL.csv'
'Parse_done/jackard_texts_test_lemm_char_ALL.csv'

'Parse_done/jackard_texts_train_lemm_word_ALL.csv'
'Parse_done/jackard_texts_test_lemm_word_ALL.csv'

'Parse_done/jackard_texts_test_lemm_word_ALL.csv'

In [47]:
X_train = pd.read_csv('Parse_done/jackard_titles_lemmatization_train_not_nan.csv', encoding='utf-8')
X_train = X_train.drop(['Unnamed: 0',
                        'max_word_1_1',
                        'max_word_2_2',
                        'max_char_3_4',
                        'mean_word_1_1',
                        'mean_word_2_2',
                        'min_word_1_1',
                        'min_char_3_4',
                        'median_word_2_2',
                        'median_char_3_4'], axis=1)
X_train.head()

,median_word_1_1,min_word_2_2,mean_char_3_4
0,0.937500,0.142857,0.902380
1,0.964286,0.533333,0.941398
2,1.000000,0.850000,0.970934
3,1.000000,1.000000,1.000000
4,1.000000,0.500000,0.930193


In [48]:
text_train_char = pd.read_csv('Parse_done/jackard_texts_train_lemm_char_ALL.csv', encoding='utf-8')
text_train_char = text_train_char.drop(['Unnamed: 0'], axis=1)
text_train_char.head()

,mean_char_3_4,median_char_3_4,min_char_3_4,max_char_3_4
0,0.953992,0.967785,0.543457,0.994654
1,0.965735,0.971823,0.858697,0.994802
2,0.977270,0.977608,0.938781,0.994800
3,0.983700,0.984179,0.970411,0.996573
4,0.964340,0.965894,0.921341,0.993499


In [49]:
text_train_word = pd.read_csv('Parse_done/jackard_texts_train_lemm_word_ALL.csv', encoding='utf-8')
text_train_word = text_train_word.drop(['Unnamed: 0'], axis=1)
text_train_word.head()

,mean_word_2_2,median_word_2_2,min_word_2_2,max_word_2_2
0,0.983877,0.998617,0.548061,1.0
1,0.993679,0.998430,0.904047,1.0
2,0.998535,0.999033,0.985533,1.0
3,0.998550,0.999086,0.990782,1.0
4,0.998143,0.998522,0.993177,1.0


In [50]:
text_raw_train_word = pd.read_csv('Parse_done/jackard_texts_train_raw_ALL.csv', encoding='utf-8')
text_raw_train_word = text_raw_train_word.drop(['Unnamed: 0'], axis=1)
text_raw_train_word.head()

,mean_char_3_4,median_char_3_4,min_char_3_4,max_char_3_4
0,0.955341,0.970104,0.510356,0.994463
1,0.969499,0.976065,0.861499,0.994004
2,0.979347,0.980122,0.947166,0.995663
3,0.985794,0.986202,0.972553,0.997323
4,0.967941,0.969287,0.931331,0.993435


In [51]:
tfidf_train = pd.read_csv('Parse_done/tfidf_train_titles_one_feat.csv', encoding='utf-8')
tfidf_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.297277,0.297277,0.260814
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.221368,0.147657,0.112685
2,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.235473,0.235473,0.164038
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.226470,0.226470,0.190008


In [52]:
svd_train = pd.read_csv('Parse_done/feature_tf-idf_svd.csv', encoding='utf-8')
svd_train.head()

,0,1
0,-2.872050e-01,-1.622207e-01
1,-1.182251e-01,2.931275e-01
2,-1.234407e-01,2.159469e-02
3,5.551115e-17,4.440892e-16
4,-2.220658e-01,-1.423748e-01


In [53]:
tfidf_train_pasha = pd.read_csv('Parse_done/simple_intersection_nearest_15TRAIN.csv', encoding='utf-8')
tfidf_train_pasha.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,6,5,4,4,3,3,3,3,3,3,3,3,2,2,2
1,1,6,6,6,6,5,5,5,4,4,3,2,2,2,2,2
2,2,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,4,4,4,3,3,3,3,3,3,3,2,2,2,2,2


In [54]:
X_train['text_mean_char_3_4'] = text_train_char['mean_char_3_4']
X_train['text_median_char_3_4'] = text_train_char['median_char_3_4']
X_train['text_min_char_3_4'] = text_train_char['min_char_3_4']
X_train['text_max_char_3_4'] = text_train_char['max_char_3_4']
X_train['text_mean_word_2_2'] = text_train_word['mean_word_2_2']
X_train['text_median_word_2_2'] = text_train_word['median_word_2_2']
X_train['text_min_word_2_2'] = text_train_word['min_word_2_2']
X_train['text_raw_mean_char_3_4'] = text_raw_train_word['mean_char_3_4']
# X_train['0'] = tfidf_train['0']
# X_train['1'] = tfidf_train['1']
# X_train['2'] = tfidf_train['2']
# X_train['3'] = tfidf_train['3']
# X_train['4'] = tfidf_train['4']
# X_train['5'] = tfidf_train['5']
# X_train['6'] = tfidf_train['6']
# X_train['7'] = tfidf_train['7']

X_train['sum_15'] = tfidf_train['sum_15']
X_train['diff_10-5'] = tfidf_train['sum_10'] - tfidf_train['sum_5']

X_train['svd0'] = svd_train['0']
X_train['svd1'] = svd_train['1']

X_train['p0'] = tfidf_train_pasha['0']
X_train['p1'] = tfidf_train_pasha['1']
X_train['p2'] = tfidf_train_pasha['2']
X_train['p3'] = tfidf_train_pasha['3']
X_train['p4'] = tfidf_train_pasha['4']
X_train['p5'] = tfidf_train_pasha['5']
X_train['p6'] = tfidf_train_pasha['6']
X_train['p7'] = tfidf_train_pasha['7']
X_train['p8'] = tfidf_train_pasha['8']
X_train['p9'] = tfidf_train_pasha['9']

X_train.head()

,median_word_1_1,min_word_2_2,mean_char_3_4,text_mean_char_3_4,text_median_char_3_4,text_min_char_3_4,text_max_char_3_4,text_mean_word_2_2,text_median_word_2_2,text_min_word_2_2,...,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,0.937500,0.142857,0.902380,0.953992,0.967785,0.543457,0.994654,0.983877,0.998617,0.548061,...,6,5,4,4,3,3,3,3,3,3
1,0.964286,0.533333,0.941398,0.965735,0.971823,0.858697,0.994802,0.993679,0.998430,0.904047,...,6,6,6,6,5,5,5,4,4,3
2,1.000000,0.850000,0.970934,0.977270,0.977608,0.938781,0.994800,0.998535,0.999033,0.985533,...,3,3,3,3,2,2,2,2,2,2
3,1.000000,1.000000,1.000000,0.983700,0.984179,0.970411,0.996573,0.998550,0.999086,0.990782,...,0,0,0,0,0,0,0,0,0,0
4,1.000000,0.500000,0.930193,0.964340,0.965894,0.921341,0.993499,0.998143,0.998522,0.993177,...,4,4,4,3,3,3,3,3,3,3


In [55]:
X_test = pd.read_csv('Parse_done/jackard_titles_lemmatization_test_not_nan.csv', encoding='utf-8')
X_test = X_test.drop(['Unnamed: 0',
                      'max_word_1_1',
                      'max_word_2_2',
                      'max_char_3_4',
                      'mean_word_1_1',
                      'mean_word_2_2',
                      'min_word_1_1',
                      'min_char_3_4',
                      'median_word_2_2',
                      'median_char_3_4'], axis=1)

X_test.shape

(16627, 3)

In [56]:
text_test_char = pd.read_csv('Parse_done/jackard_texts_test_lemm_char_ALL.csv', encoding='utf-8')
text_test_char = text_test_char.drop(['Unnamed: 0'], axis=1)
text_test_char.head()

,mean_char_3_4,median_char_3_4,min_char_3_4,max_char_3_4
0,0.957246,0.968203,0.615230,0.992755
1,0.968171,0.967919,0.946627,0.988069
2,0.963468,0.962889,0.935294,0.985657
3,0.973880,0.974538,0.913814,0.995257
4,0.979027,0.979619,0.956600,0.995885


In [57]:
text_test_word = pd.read_csv('Parse_done/jackard_texts_test_lemm_word_ALL.csv', encoding='utf-8')
text_test_word = text_test_word.drop(['Unnamed: 0'], axis=1)
text_test_word.head()

,mean_word_2_2,median_word_2_2,min_word_2_2,max_word_2_2
0,0.985816,0.998669,0.673469,1.0
1,0.997769,0.998494,0.987935,1.0
2,0.996283,0.997729,0.980564,1.0
3,0.997155,0.999114,0.968397,1.0
4,0.999324,0.999412,0.996477,1.0


In [58]:
text_raw_test_word = pd.read_csv('Parse_done/jackard_texts_test_raw_ALL.csv', encoding='utf-8')
text_raw_test_word = text_raw_test_word.drop(['Unnamed: 0'], axis=1)
text_raw_test_word.head()

,mean_char_3_4,median_char_3_4,min_char_3_4,max_char_3_4
0,0.960963,0.971544,0.597319,0.993071
1,0.969868,0.970075,0.950363,0.988753
2,0.964305,0.963249,0.939213,0.989073
3,0.975029,0.974124,0.923479,0.995526
4,0.979440,0.981631,0.954598,0.996291


In [59]:
X_test['text_mean_char_3_4'] = text_test_char['mean_char_3_4']
X_test['text_median_char_3_4'] = text_test_char['median_char_3_4']
X_test['text_min_char_3_4'] = text_test_char['min_char_3_4']
X_test['text_max_char_3_4'] = text_test_char['max_char_3_4']
X_test['text_mean_word_2_2'] = text_test_word['mean_word_2_2']
X_test['text_median_word_2_2'] = text_test_word['median_word_2_2']
X_test['text_min_word_2_2'] = text_test_word['min_word_2_2']
X_test['text_raw_mean_char_3_4'] = text_raw_test_word['mean_char_3_4']

X_test.shape

(16627, 11)

In [60]:
tfidf_test = pd.read_csv('Parse_done/tfidf_test_titles_one_feat.csv', encoding='utf-8')
tfidf_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,sum_15,sum_10,sum_5
0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.203793,0.187374,0.151433
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040930,0.040930,0.040930
2,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187374,0.187374,0.151433
3,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.187374,0.187374,0.151433
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.088781,0.088781,0.040930


In [61]:
svd_test = pd.read_csv('Parse_done/feature_tf-idf_svd_TEST.csv', encoding='utf-8')
svd_test.head()

,0,1
0,-0.199917,-0.158953
1,-0.039768,0.176584
2,-0.127357,-0.074872
3,-0.262177,-0.203920
4,-0.031422,0.186940


In [62]:
tfidf_test_pasha = pd.read_csv('Parse_done/simple_intersection_nearest_15.csv', encoding='utf-8')
tfidf_test_pasha.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,5,5,5,5,5,4,4,3,3,3,3,3,3,3,3
1,1,3,2,2,2,2,2,2,2,2,2,1,1,1,1,1
2,2,5,5,5,5,4,4,4,4,3,3,3,3,3,3,3
3,3,4,4,4,4,3,3,3,3,3,3,3,3,3,2,2
4,4,3,2,2,1,1,1,1,1,1,1,1,1,1,1,1


In [63]:
# X_test['0'] = tfidf_test['0']
# X_test['1'] = tfidf_test['1']
# X_test['2'] = tfidf_test['2']
# X_test['3'] = tfidf_test['3']
# X_test['4'] = tfidf_test['4']
# X_test['5'] = tfidf_test['5']
# X_test['6'] = tfidf_test['6']
# X_test['7'] = tfidf_test['7']

X_test['sum_15'] = tfidf_test['sum_15']
X_test['diff_10-5'] = tfidf_test['sum_10'] - tfidf_test['sum_5']

X_test['svd0'] = svd_test['0']
X_test['svd1'] = svd_test['1']

X_test['p0'] = tfidf_test_pasha['0']
X_test['p1'] = tfidf_test_pasha['1']
X_test['p2'] = tfidf_test_pasha['2']
X_test['p3'] = tfidf_test_pasha['3']
X_test['p4'] = tfidf_test_pasha['4']
X_test['p5'] = tfidf_test_pasha['5']
X_test['p6'] = tfidf_test_pasha['6']
X_test['p7'] = tfidf_test_pasha['7']
X_test['p8'] = tfidf_test_pasha['8']
X_test['p9'] = tfidf_test_pasha['9']

In [64]:
X_test.describe()

,median_word_1_1,min_word_2_2,mean_char_3_4,text_mean_char_3_4,text_median_char_3_4,text_min_char_3_4,text_max_char_3_4,text_mean_word_2_2,text_median_word_2_2,text_min_word_2_2,...,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
count,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,...,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000
mean,0.955268,0.681376,0.940280,0.962184,0.963589,0.874904,0.994359,0.995120,0.997605,0.919981,...,2.774223,2.259277,2.025561,1.871053,1.745534,1.641066,1.558429,1.482288,1.418235,1.354604
std,0.087359,0.272764,0.087786,0.014266,0.013941,0.154991,0.006198,0.007645,0.001961,0.174016,...,2.242601,1.943216,1.849181,1.792387,1.743893,1.701180,1.665451,1.633721,1.607258,1.574052
min,0.500000,0.000000,0.490244,0.827880,0.918325,0.000000,0.960490,0.850120,0.982409,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.937500,0.526316,0.923710,0.954784,0.955134,0.898144,0.992093,0.994785,0.996802,0.953680,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.769231,0.981516,0.963769,0.963828,0.927763,0.996529,0.997141,0.998139,0.986870,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,0.888889,0.995362,0.971346,0.972444,0.940477,0.998799,0.998300,0.998971,0.992658,...,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,29.000000,19.000000,18.000000,18.000000,16.000000,15.000000,15.000000,13.000000,12.000000,12.000000


In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint as randint

In [66]:
val_ = 5
cv = range(1, val_ + 1)
group_in_val = train_groups.iloc[-1] // val_

n_estimators = [10, 20, 30, 40, 50,
                60, 70, 80, 90, 100]
min_samples_split = [5, 6, 7, 8, 9, 10]
max_depth = [2, 3, 4, 5, 6, 7, 8]

aver_score_log = []

for max_depth_ in max_depth:
    for min_samples_split_ in min_samples_split:
        for n_estimators_ in n_estimators:
            score_log = np.zeros(len(cv))
            for i in cv:
                mask_l = train_groups <= group_in_val * (i - 1)
                mask_r = train_groups > group_in_val * i
                X_val_train = X_train[mask_l | mask_r]
                y_val_train = train_y[mask_l | mask_r]
                model_cv = RandomForestClassifier(random_state=123,
                                                  n_estimators=n_estimators_,
                                                  n_jobs=-1,
                                                  max_depth=max_depth_,
                                                  min_samples_split=min_samples_split_)
                model_cv.fit(X_val_train, y_val_train)

                mask_l = ~mask_l
                mask_r = ~mask_r
                X_val = X_train[mask_l | mask_r]
                y_val = train_y[mask_l | mask_r]
                score_log[i-1] = f1_score(y_pred=model_cv.predict(X_val),
                                          y_true=y_val, average='macro')
            aver_score_log.append(score_log.mean())

aver_score_log = np.array(aver_score_log)
opt_ind = aver_score_log.argmax()
print('rf_max: ', aver_score_log.max(), opt_ind)

rf_max:  0.8739621281146718 367


In [67]:
max_depth_optimal_log = max_depth[opt_ind // (len(min_samples_split) * len(n_estimators)) % len(max_depth)]
min_samples_split_optimal_log = min_samples_split[opt_ind // len(n_estimators) % len(min_samples_split)]
n_estimators_optimal_log = n_estimators[opt_ind % len(n_estimators)]
print('max_depth={0}, min_samples={1}, n_estimators={2}'.format(max_depth_optimal_log,
                                               min_samples_split_optimal_log,
                                               n_estimators_optimal_log))

max_depth=8, min_samples=5, n_estimators=80


In [68]:
model_titles_lem = RandomForestClassifier(random_state=123,
                                          n_estimators=n_estimators_optimal_log,
                                          n_jobs=-1,
                                          max_depth=max_depth_optimal_log,
                                          min_samples_split=min_samples_split_optimal_log)
model_titles_lem.fit(X_train, train_y)
f1_score(y_pred=model_titles_lem.predict(X_train), y_true=train_y, average='macro')

0.8795454508412138

In [69]:
y_pred = model_titles_lem.predict(X_test)
with open('rand_forest_titles+tfidf+svd.csv', 'w') as f:
    print("pair_id,target", file=f)
    for pair, y in zip(test_data.pair_id, y_pred):
        print("{0},{1}".format(pair, y), file=f)

In [70]:
y_pred.mean()

0.24147470980934624

In [71]:
model_titles_lem.feature_importances_

array([0.11069266, 0.02632898, 0.14487935, 0.0450399 , 0.02428168,
       0.01855038, 0.01514969, 0.02051277, 0.03425171, 0.01449776,
       0.02140173, 0.15439798, 0.00841343, 0.03976017, 0.01516909,
       0.00974745, 0.00893736, 0.02930214, 0.07106518, 0.03054323,
       0.01335379, 0.02436103, 0.05722389, 0.04516778, 0.01697084])

In [72]:
X_test.describe()

,median_word_1_1,min_word_2_2,mean_char_3_4,text_mean_char_3_4,text_median_char_3_4,text_min_char_3_4,text_max_char_3_4,text_mean_word_2_2,text_median_word_2_2,text_min_word_2_2,...,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
count,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,...,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000,16627.000000
mean,0.955268,0.681376,0.940280,0.962184,0.963589,0.874904,0.994359,0.995120,0.997605,0.919981,...,2.774223,2.259277,2.025561,1.871053,1.745534,1.641066,1.558429,1.482288,1.418235,1.354604
std,0.087359,0.272764,0.087786,0.014266,0.013941,0.154991,0.006198,0.007645,0.001961,0.174016,...,2.242601,1.943216,1.849181,1.792387,1.743893,1.701180,1.665451,1.633721,1.607258,1.574052
min,0.500000,0.000000,0.490244,0.827880,0.918325,0.000000,0.960490,0.850120,0.982409,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.937500,0.526316,0.923710,0.954784,0.955134,0.898144,0.992093,0.994785,0.996802,0.953680,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.769231,0.981516,0.963769,0.963828,0.927763,0.996529,0.997141,0.998139,0.986870,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,0.888889,0.995362,0.971346,0.972444,0.940477,0.998799,0.998300,0.998971,0.992658,...,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,29.000000,19.000000,18.000000,18.000000,16.000000,15.000000,15.000000,13.000000,12.000000,12.000000
